# San_Diego Crossing: Guatemala
- Four SSP Scenarios
- Saving output DF and Graphs

In [ ]:
# Libraries
import pandas as pd
import numpy as np
import matplotlib.pylab as plt

# Imputting libraries
from sklearn.linear_model import LinearRegression

# Saving Model Summaries
import statsmodels.api as sm

# Formatting printing and floats
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:.2f}'.format

# Parsing into a dataframe
wide_df = pd.read_csv('INPUTS_OUTPUTS_NEW_VARIABLES_ML/INPUT_DATA_SCRIPT_ML/Guatemala/GUA_USA_FINAL_VARS.csv')

# Dropping columns
cols_out = ['CSV', 'Country', 'New_Data_Type']
wide_df = wide_df.drop(cols_out, axis=1).copy()

# Cleaning/parsing
wide_df.iloc[:, 4:] = wide_df.iloc[:, 4:].replace(0, np.nan)  # Replacing all 0 values with NaN values
wide_df.iloc[:, 4:] = wide_df.iloc[:, 4:].astype(float)       # Data type to float 

In [ ]:
wide_df

In [ ]:
# Function to parse data into numeric

def prepare_data(df, year_columns):
    df[year_columns] = df[year_columns].apply(pd.to_numeric, errors='coerce')
    return df

# # Function to impute data with OLS

def impute_with_ols(df, train_start_year, train_end_year):
    # Defining columns
    year_columns = [str(year) for year in range(train_start_year, train_end_year + 1)]
    # Imputting data
    for index, row in df.iterrows():
        # Fitting the model only on available data
        available_data = row[year_columns].dropna()
        if len(available_data) < 2:
            continue  # Need at least two data points to fit a line
        
        # Determining imputation method for each variable
        impute_method = {}
        for col in available_data.index:
            if col in ['GDP (current US$)', 'GDP per capita (current US$)', 'Unemployment, total (% of total labor force) (modeled ILO estimate)']:
                impute_method[col] = 'mean'
            else:
                impute_method[col] = 'median'

        # Setting variables for OLS
        X_train = np.array(list(map(int, available_data.index))).reshape(-1, 1)
        y_train = available_data.values
        model = LinearRegression()
        model.fit(X_train, y_train)
        
        # Predicting missing values
        missing_years = row[year_columns][row[year_columns].isna()].index
        if missing_years.empty:
            continue
        X_missing = np.array(list(map(int, missing_years))).reshape(-1, 1)
        predicted_values = model.predict(X_missing)
        
        # Filling missing values in the DataFrame using the appropriate imputation method
        for col in missing_years:
            if col in impute_method:
                if impute_method[col] == 'mean':
                    df.loc[index, col] = row[year_columns].mean()
                elif impute_method[col] == 'median':
                    df.loc[index, col] = row[year_columns].median()

    return df


# Converting year columns to numeric: 1960 - 2100

year_columns = [str(year) for year in range(1960, 2100)]
wide_df = prepare_data(wide_df, year_columns)

In [ ]:
wide_df.tail()

In [ ]:
# OLS : 2015 TO 2022

# Perform imputation with OLS for specified years
wide_df_processed_data_ols = impute_with_ols(wide_df, 2015, 2022)
wide_df_processed_data_ols = wide_df_processed_data_ols

# Dropping columns
years_to_drop = [str(year) for year in range(1960, 2015)]
wide_df_processed_data_ols = wide_df_processed_data_ols.drop(columns=years_to_drop)
# wide_df_processed_data_ols.head()

In [ ]:
# Function to parse data into numeric

def convert_to_numeric(df):
    for col in df.columns:
        try:
            # Force convert to float and handle exceptions
            df[col] = pd.to_numeric(df[col], errors='raise')
        except ValueError as e:
            # Log columns that could not be converted, with error message
            print(f"Column {col} cannot be converted to numeric: {e}")
        except Exception as e:
            # Log unexpected exceptions
            print(f"Unexpected error with column {col}: {e}")
    return df

# Function for Linear Interpolation

def linear_interpolation2(df, start_year, end_year):
    # Create a list of year columns
    year_columns = [str(year) for year in range(start_year, end_year + 1)]
    # Filter out columns that are not in the year range
    year_columns = [col for col in year_columns if col in df.columns]

    # Convert all potential year columns to numeric
    df[year_columns] = convert_to_numeric(df[year_columns])

    # Apply linear interpolation to only the year columns
    df[year_columns] = df[year_columns].interpolate(method='linear', axis=1, limit_direction='both')

    return df

# Calling function with specified years

wide_df_processed_data_ols = linear_interpolation2(wide_df_processed_data_ols, 1960, 2022)
# wide_df_processed_data_ols

In [ ]:
# wide_df_processed_data_ols['Type__of_Variable'][7:]

In [ ]:
# Melting the dataframe

long_df = pd.melt(wide_df_processed_data_ols, id_vars=['Country_Code', 'Variable', 
                                                       'Unit_of_Measure', 'Type_of_Variable'],
                                                         var_name='Year', value_name='Value')

# Printing melted dataframe 
long_df['Year'] = long_df['Year'].astype(int) 
long_df_22to100 = long_df[long_df['Year'] >= 2021]

In [ ]:
long_df.head(100)

In [ ]:
# Libraries

import statsmodels.api as sm
# from sklearn.datasets import load_iris 
from math import log
import statsmodels.api as sm
# from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Filter data for temperature projections and historical data

# Historical Data from 1960 to 2022
historical_data = long_df[long_df['Year'] <= 2022]
# Predicted Data from 2022 to 2100
predicted_data = long_df[long_df['Year'] >= 2023]

In [ ]:
##################################################################

# Scenario 1: GUA_SSP119_avg_temp
# features_1960_2022_ssp119

# Creating a dataframe 'features_1960_2022_ssp119' 
# Years 1960 to 2022
# 'features_ssp119' stores all values from all variables except SSP and TRAC data
# 'features_ssp119' = X

# Filtering data for 'GUA_SSP119_avg_temp'
GUA_features_1960_2022_ssp119 = historical_data[historical_data['Variable'] == 'GUA_SSP119_avg_temp'] 
GUA_features_1960_2022_ssp119 = GUA_features_1960_2022_ssp119[['Year', 'Value']]                            
GUA_features_1960_2022_ssp119 = GUA_features_1960_2022_ssp119.rename(columns={'Value': 'Temperature'})      

##################################################################

# Scenario 2: GUA_SSP245_avg_temp
# features_1960_2022_ssp245

# Creating a dataframe 'features_1960_2022_ssp245' 
# Years 1960 to 2022
# 'features_ssp119' stores all values from all variables except SSP and TRAC data
# 'features_ssp119' = X

# Filtering data for 'GUA_SSP245_avg_temp'
GUA_features_1960_2022_ssp245 = historical_data[historical_data['Variable'] == 'GUA_SSP245_avg_temp'] 
GUA_features_1960_2022_ssp245 = GUA_features_1960_2022_ssp245[['Year', 'Value']]                            
GUA_features_1960_2022_ssp245 = GUA_features_1960_2022_ssp245.rename(columns={'Value': 'Temperature'})      

##################################################################

# Scenario 3: GUA_SSP370_avg_temp
# features_1960_2022_ssp370

# Creating a dataframe 'features_1960_2022_ssp370' 
# Years 1960 to 2022
# 'features_ssp119' stores all values from all variables except SSP and TRAC data
# 'features_ssp119' = X

# Filtering data for 'GUA_SSP245_avg_temp'
GUA_features_1960_2022_ssp370 = historical_data[historical_data['Variable'] == 'GUA_SSP370_avg_temp'] 
GUA_features_1960_2022_ssp370 = GUA_features_1960_2022_ssp370[['Year', 'Value']]                            
GUA_features_1960_2022_ssp370 = GUA_features_1960_2022_ssp370.rename(columns={'Value': 'Temperature'})      

##################################################################

# Scenario 4: GUA_SSP585_avg_temp
# features_1960_2022_ssp585

# Creating a dataframe 'features_1960_2022_ssp585' 
# Years 1960 to 2022
# 'features_ssp119' stores all values from all variables except SSP and TRAC data
# 'features_ssp119' = X

# Filtering data for 'GUA_SSP585_avg_temp'
GUA_features_1960_2022_ssp585 = historical_data[historical_data['Variable'] == 'GUA_SSP585_avg_temp'] 
GUA_features_1960_2022_ssp585 = GUA_features_1960_2022_ssp585[['Year', 'Value']]                            
GUA_features_1960_2022_ssp585 = GUA_features_1960_2022_ssp585.rename(columns={'Value': 'Temperature'})      

##################################################################

# Additional Variables to all models
# additional_variables: not considering SSP and TRAC data: X
additional_variables = [
    'Population, total',
    'Rural population (% of total population)',
    'Population ages 15-64 (% of total population)',
    'Unemployment, total (% of total labor force) (modeled ILO estimate)',
    'Victims of intentional homicide',
    'Prevalence of severe food insecurity in the population (%)',
    'Prevalence of undernourishment (% of population)',
    'Corruption',
    'Government Effectiveness: Estimate',
    'GDP (current US$)',
    'GDP per capita (current US$)'
]
    
##################################################################

##################################################################

# Scenario 1: _SSP119_avg_temp
# features_1960_2022_ssp119
# Getting all values from additional_variables into features
for var in additional_variables:
    var_data = historical_data[historical_data['Variable'] == var][['Year', 'Value']]
    var_data = var_data.rename(columns={'Value': var})
    GUA_features_1960_2022_ssp119 = pd.merge(GUA_features_1960_2022_ssp119, var_data, on='Year', how='outer') 
    
# features_1960_2022_ssp119.tail() 
##################################################################

# Scenario 2: _SSP245_avg_temp
# features_1960_2022_ssp245
# Getting all values from additional_variables into features
for var in additional_variables:
    var_data = historical_data[historical_data['Variable'] == var][['Year', 'Value']]
    var_data = var_data.rename(columns={'Value': var})
    GUA_features_1960_2022_ssp245 = pd.merge(GUA_features_1960_2022_ssp245, var_data, on='Year', how='outer') #CHANGE!!!!!!!!!!!
    
# features_1960_2022_ssp245.tail() 
##################################################################

# Scenario 3: _SSP370_avg_temp
# features_1960_2022_ssp370
# Getting all values from additional_variables into features
for var in additional_variables:
    var_data = historical_data[historical_data['Variable'] == var][['Year', 'Value']]
    var_data = var_data.rename(columns={'Value': var})
    GUA_features_1960_2022_ssp370 = pd.merge(GUA_features_1960_2022_ssp370, var_data, on='Year', how='outer') #CHANGE!!!!!!!!!!!
    
# features_1960_2022_ssp370.tail() 
##################################################################

# Scenario 4: _SSP585_avg_temp
# features_1960_2022_ssp585
# Getting all values from additional_variables into features
for var in additional_variables:
    var_data = historical_data[historical_data['Variable'] == var][['Year', 'Value']]
    var_data = var_data.rename(columns={'Value': var})
    GUA_features_1960_2022_ssp585 = pd.merge(GUA_features_1960_2022_ssp585, var_data, on='Year', how='outer') #CHANGE!!!!!!!!!!!

# features_1960_2022_ssp585.head() 

In [ ]:
# GUA_features_1960_2022_ssp119

In [ ]:
# Libraries 

from sklearn.linear_model import Ridge
from sklearn.multioutput import MultiOutputRegressor

In [ ]:
# Historic data on temperature
X1_119_GUA = GUA_features_1960_2022_ssp119[['Temperature']]         
# Predictions on training data based on temperature
Y1_119_GUA = GUA_features_1960_2022_ssp119.drop(columns='Temperature')      
# Training model with Ridge Regression for 'GUA_SSP119_avg_temp' based on 'Temperature'
model119_GUA = MultiOutputRegressor(Ridge(random_state=123)).fit(X1_119_GUA, Y1_119_GUA)

In [ ]:
# Y1_119_GUA.shape    # Y1_119_GUA.shape # (32768, 14)
# X1_119_GUA.shape # Y1_119_GUA.shape # (32768, 1)
Y1_119_GUA.tail()

In [ ]:
model119_GUA

In [ ]:
##################################################################
# Training Data: training all features based on temperature

######### Crossing 1: GUA_San_Diego #########
# X1 and Y1 are based on features_ssp119 which was filtered with 'GUA_SSP119_avg_temp'

# Historic data on temperature
X1_119_GUA = GUA_features_1960_2022_ssp119[['Temperature']]         
# Predictions on training data based on temperature
Y1_119_GUA = GUA_features_1960_2022_ssp119.drop(columns='Temperature')      
# Training model with Ridge Regression for 'GUA_SSP119_avg_temp' based on 'Temperature'
model119_GUA = MultiOutputRegressor(Ridge(random_state=123)).fit(X1_119_GUA, Y1_119_GUA)

##################################################################
##################################################################
# Training Data: training all features based on temperature

######### Crossing 1: GUA_San_Diego #########
# X1 and Y1 are based on features_ssp245 which was filtered with 'GUA_SSP245_avg_temp'

# Historic data on temperature
X2_245_GUA = GUA_features_1960_2022_ssp245[['Temperature']]         
# Predictions on training data based on temperature
Y2_245_GUA = GUA_features_1960_2022_ssp245.drop(columns='Temperature')      
# Training model with Ridge Regression for 'GUA_SSP119_avg_temp' based on 'Temperature'
model245_GUA = MultiOutputRegressor(Ridge(random_state=123)).fit(X2_245_GUA, Y2_245_GUA)

##################################################################
##################################################################
# Training Data: training all features based on temperature

######### Crossing 1: GUA_San_Diego #########
# X1 and Y1 are based on features_ssp370 which was filtered with 'GUA_SSP370_avg_temp'

# Historic data on temperature
X3_370_GUA = GUA_features_1960_2022_ssp370[['Temperature']]         
# Predictions on training data based on temperature
Y3_370_GUA = GUA_features_1960_2022_ssp370.drop(columns='Temperature')      
# Training model with Ridge Regression for 'GUA_SSP119_avg_temp' based on 'Temperature'
model370_GUA = MultiOutputRegressor(Ridge(random_state=123)).fit(X3_370_GUA, Y3_370_GUA)

##################################################################
##################################################################
# Training Data: training all features based on temperature

######### Crossing 1: GUA_San_Diego #########
# X1 and Y1 are based on features_ssp585 which was filtered with 'GUA_SSP585_avg_temp'

# Historic data on temperature
X4_585_GUA = GUA_features_1960_2022_ssp585[['Temperature']]         
# Predictions on training data based on temperature
Y4_585_GUA = GUA_features_1960_2022_ssp585.drop(columns='Temperature')      
# Training model with Ridge Regression for 'GUA_SSP119_avg_temp' based on 'Temperature'
model585_GUA = MultiOutputRegressor(Ridge(random_state=123)).fit(X4_585_GUA, Y4_585_GUA)

##################################################################

In [ ]:
# Prediction Dataframe: 2023 to 2100 (predicted_data)
##################################################################

# Additional Variables to all models
# additional_variables: not considering SSP and TRAC data: X
#####

additional_variables = [
    'Population, total',
    'Rural population (% of total population)',
    'Population ages 15-64 (% of total population)',
    'Unemployment, total (% of total labor force) (modeled ILO estimate)',
    'Victims of intentional homicide',
    'Prevalence of severe food insecurity in the population (%)',
    'Prevalence of undernourishment (% of population)',
    'Corruption',
    'Government Effectiveness: Estimate',
    'GDP (current US$)',
    'GDP per capita (current US$)'
]

#####


# Scenario 1: _SSP119_avg_temp = Y
##################################################################
# Creating a dataframe 'new_features_2023_2100_ssp119' 
# Years 2023 to 2100
# 'features_ssp119' stores all values from all variables except SSP and TRAC data
# 'new_features_2023_2100_ssp119' = X
#####
# Filtering data for 'GUA_SSP119_avg_temp'
GUA_new_features_2023_2100_ssp119 = predicted_data[predicted_data['Variable'] == 'GUA_SSP119_avg_temp']
GUA_new_features_2023_2100_ssp119 = GUA_new_features_2023_2100_ssp119[['Year', 'Value']]  
GUA_new_features_2023_2100_ssp119 = GUA_new_features_2023_2100_ssp119.rename(columns={'Value': 'Temperature'})  
#####
# Loop through each additional variable and merge with X1 DataFrame
# _SSP119_avg_temp
for var in additional_variables:
    var_data = predicted_data[predicted_data['Variable'] == var][['Year', 'Value']]
    var_data = var_data.rename(columns={'Value': var})
    GUA_new_features_2023_2100_ssp119 = pd.merge(GUA_new_features_2023_2100_ssp119, var_data, on='Year', how='outer')
##################################################################


# Scenario 2: _SSP245_avg_temp = Y
##################################################################
# Creating a dataframe 'new_features_2023_2100_ssp245' 
# Years 2023 to 2100
# 'features_ssp119' stores all values from all variables except SSP and TRAC data
# 'new_features_2023_2100_ssp245' = X
#####
# Filtering data for 'GUA_SSP245_avg_temp'
GUA_new_features_2023_2100_ssp245 = predicted_data[predicted_data['Variable'] == 'GUA_SSP245_avg_temp']
GUA_new_features_2023_2100_ssp245 = GUA_new_features_2023_2100_ssp245[['Year', 'Value']]  
GUA_new_features_2023_2100_ssp245 = GUA_new_features_2023_2100_ssp245.rename(columns={'Value': 'Temperature'})  
#####
# Loop through each additional variable and merge with X1 DataFrame
# _SSP245_avg_temp
for var in additional_variables:
    var_data = predicted_data[predicted_data['Variable'] == var][['Year', 'Value']]
    var_data = var_data.rename(columns={'Value': var})
    GUA_new_features_2023_2100_ssp245 = pd.merge(GUA_new_features_2023_2100_ssp245, var_data, on='Year', how='outer')
##################################################################


# Scenario 3: _SSP370_avg_temp = Y
##################################################################
# Creating a dataframe 'new_features_2023_2100_ssp370' 
# Years 2023 to 2100
# 'features_ssp119' stores all values from all variables except SSP and TRAC data
# 'new_features_2023_2100_ssp370' = X

# Filtering data for 'GUA_SSP370_avg_temp'
GUA_new_features_2023_2100_ssp370 = predicted_data[predicted_data['Variable'] == 'GUA_SSP370_avg_temp']
GUA_new_features_2023_2100_ssp370 = GUA_new_features_2023_2100_ssp370[['Year', 'Value']]  
GUA_new_features_2023_2100_ssp370 = GUA_new_features_2023_2100_ssp370.rename(columns={'Value': 'Temperature'})  
#####
# Loop through each additional variable and merge with X1 DataFrame
# _SSP370_avg_temp
for var in additional_variables:
    var_data = predicted_data[predicted_data['Variable'] == var][['Year', 'Value']]
    var_data = var_data.rename(columns={'Value': var})
    GUA_new_features_2023_2100_ssp370 = pd.merge(GUA_new_features_2023_2100_ssp370, var_data, on='Year', how='outer')
##################################################################


# Scenario 4: _SSP585_avg_temp = Y
##################################################################
# Creating a dataframe 'new_features_2023_2100_ssp585' 
# Years 2023 to 2100
# 'features_ssp119' stores all values from all variables except SSP and TRAC data
# 'new_features_2023_2100_ssp585' = X

# Filtering data for 'GUA_SSP585_avg_temp'
GUA_new_features_2023_2100_ssp585 = predicted_data[predicted_data['Variable'] == 'GUA_SSP585_avg_temp']
GUA_new_features_2023_2100_ssp585 = GUA_new_features_2023_2100_ssp585[['Year', 'Value']]  
GUA_new_features_2023_2100_ssp585 = GUA_new_features_2023_2100_ssp585.rename(columns={'Value': 'Temperature'})  
#####
# Loop through each additional variable and merge with X1 DataFrame
# _SSP585_avg_temp
for var in additional_variables:
    var_data = predicted_data[predicted_data['Variable'] == var][['Year', 'Value']]
    var_data = var_data.rename(columns={'Value': var})
    GUA_new_features_2023_2100_ssp585 = pd.merge(GUA_new_features_2023_2100_ssp585, var_data, on='Year', how='outer')

In [ ]:
# Predicting all NaN values of the additional_variables

##################################################################
# X2 is getting just the temperature values
# new_features_2023_2100_ssp119
X2_119_GUA = GUA_new_features_2023_2100_ssp119[['Temperature']]
# Here the model is predicting all values and storing them in Y2
Y2_119_GUA = model119_GUA.predict(X2_119_GUA)[:,1:]
# The additional_variables are now getting the predicted Y2 values
GUA_new_features_2023_2100_ssp119[additional_variables] = Y2_119_GUA
##################################################################
# X2 is getting just the temperature values
# new_features_2023_2100_ssp245
X2_245_GUA = GUA_new_features_2023_2100_ssp245[['Temperature']]
# Here the model is predicting all values and storing them in Y2
Y2_245_GUA = model245_GUA.predict(X2_245_GUA)[:,1:]
# The additional_variables are now getting the predicted Y2 values
GUA_new_features_2023_2100_ssp245[additional_variables] = Y2_245_GUA
##################################################################
# X2 is getting just the temperature values
# new_features_2023_2100_ssp370
X2_370_GUA = GUA_new_features_2023_2100_ssp370[['Temperature']]
# Here the model is predicting all values and storing them in Y2
Y2_370_GUA = model370_GUA.predict(X2_370_GUA)[:,1:]
# The additional_variables are now getting the predicted Y2 values
GUA_new_features_2023_2100_ssp370[additional_variables] = Y2_370_GUA
##################################################################
# X2 is getting just the temperature values
# new_features_2023_2100_ssp585
X2_585_GUA = GUA_new_features_2023_2100_ssp585[['Temperature']]
# Here the model is predicting all values and storing them in Y2
Y2_585_GUA = model585_GUA.predict(X2_585_GUA)[:,1:]
# The additional_variables are now getting the predicted Y2 values
GUA_new_features_2023_2100_ssp585[additional_variables] = Y2_585_GUA

# Predicting TRAC from all features

In [ ]:
# Filter data for temperature projections and historical data

# Prediction starts from 2015 because TRAC starts on 2015
# long_df goes from 2015 to 2100

# Temperature Data from 2023 to 2100, all four SSPs

# Historical Data from 2015 to 2022
long_hist_data = long_df[long_df['Year'] <= 2022]
# Predicted Data from 2023 to 2100
long_predicted_data = long_df[long_df['Year'] >= 2023]
# Dependent Variables, TRAC Crossings
dep_vars = long_hist_data[long_hist_data['Variable'].isin(['GUA_Rio_Grande', 'GUA_San_Diego', 'GUA_San_Diego'])]

In [ ]:
##################################################################
# Filter data for 'GUA_SSP119_avg_temp'

X1_119_GUA = long_hist_data[long_hist_data['Variable'] == 'GUA_SSP119_avg_temp']
X1_119_GUA = X1_119_GUA[['Year', 'Value']]
X1_119_GUA = X1_119_GUA.rename(columns={'Value': 'Temperature'})
##################################################################
# Filter data for 'GUA_SSP245_avg_temp'

X1_245_GUA = long_hist_data[long_hist_data['Variable'] == 'GUA_SSP245_avg_temp']
X1_245_GUA = X1_245_GUA[['Year', 'Value']]
X1_245_GUA = X1_245_GUA.rename(columns={'Value': 'Temperature'})
##################################################################
# Filter data for 'GUA_SSP370_avg_temp'

X1_370_GUA = long_hist_data[long_hist_data['Variable'] == 'GUA_SSP370_avg_temp']
X1_370_GUA = X1_370_GUA[['Year', 'Value']]
X1_370_GUA = X1_370_GUA.rename(columns={'Value': 'Temperature'})
##################################################################
# Filter data for 'GUA_SSP585_avg_temp'

X1_585_GUA = long_hist_data[long_hist_data['Variable'] == 'GUA_SSP585_avg_temp']
X1_585_GUA = X1_585_GUA[['Year', 'Value']]
X1_585_GUA = X1_585_GUA.rename(columns={'Value': 'Temperature'})
##################################################################

# List of additional variables to add to X1

additional_variables = [
    'Population, total',
    'Rural population (% of total population)',
    'Population ages 15-64 (% of total population)',
    'Unemployment, total (% of total labor force) (modeled ILO estimate)',
    'Victims of intentional homicide',
    'Prevalence of severe food insecurity in the population (%)',
    'Prevalence of undernourishment (% of population)',
    'Corruption',
    'Government Effectiveness: Estimate',
    'GDP (current US$)',
    'GDP per capita (current US$)',
    'GUA_San_Diego'
]


##################################################################
# Loop through each additional variable and merge with X1 DataFrame
for var in additional_variables:
    var_data = long_hist_data[long_hist_data['Variable'] == var][['Year', 'Value']]
    var_data = var_data.rename(columns={'Value': var})
    X1_119_GUA = pd.merge(X1_119_GUA, var_data, on='Year', how='outer')
# Now X1 contains 'Temperature' along with the additional variables for each year
# 2015 to 2022
##################################################################
# Loop through each additional variable and merge with X1 DataFrame
for var in additional_variables:
    var_data = long_hist_data[long_hist_data['Variable'] == var][['Year', 'Value']]
    var_data = var_data.rename(columns={'Value': var})
    X1_245_GUA = pd.merge(X1_245_GUA, var_data, on='Year', how='outer')
# Now X1 contains 'Temperature' along with the additional variables for each year
# 2015 to 2022
##################################################################
# Loop through each additional variable and merge with X1 DataFrame
for var in additional_variables:
    var_data = long_hist_data[long_hist_data['Variable'] == var][['Year', 'Value']]
    var_data = var_data.rename(columns={'Value': var})
    X1_370_GUA = pd.merge(X1_370_GUA, var_data, on='Year', how='outer')
# Now X1 contains 'Temperature' along with the additional variables for each year
# 2015 to 2022
##################################################################
# Loop through each additional variable and merge with X1 DataFrame
for var in additional_variables:
    var_data = long_hist_data[long_hist_data['Variable'] == var][['Year', 'Value']]
    var_data = var_data.rename(columns={'Value': var})
    X1_585_GUA = pd.merge(X1_585_GUA, var_data, on='Year', how='outer')
# Now X1 contains 'Temperature' along with the additional variables for each year

In [ ]:
##################################################################
# _SSP119_avg_temp
X1_119_new_GUA = X1_119_GUA[X1_119_GUA['Year'] >= 2015]
##################################################################
# _SSP245_avg_temp
X1_245_new_GUA = X1_245_GUA[X1_245_GUA['Year'] >= 2015]
##################################################################
# _SSP370_avg_temp
X1_370_new_GUA = X1_370_GUA[X1_370_GUA['Year'] >= 2015]
##################################################################
# _SSP585_avg_temp
X1_585_new_GUA = X1_585_GUA[X1_585_GUA['Year'] >= 2015]

In [ ]:
# X1_119_new_GUA.head()

# Predicting San_Diego Crossings

In [ ]:
##################################################################
# Predicting Y: _SSP119_avg_temp
Y1_119_GUA = X1_119_new_GUA[['GUA_San_Diego']]
X1_predict_119_GUA = X1_119_new_GUA.drop(columns='GUA_San_Diego')
##################################################################
# Predicting Y: _SSP245_avg_temp
Y1_245_GUA = X1_245_new_GUA[['GUA_San_Diego']]
X1_predict_245_GUA = X1_245_new_GUA.drop(columns='GUA_San_Diego')
##################################################################
# Predicting Y: _SSP370_avg_temp
Y1_370_GUA = X1_370_new_GUA[['GUA_San_Diego']]
X1_predict_370_GUA = X1_370_new_GUA.drop(columns='GUA_San_Diego')
##################################################################
# Predicting Y: _SSP585_avg_temp
Y1_585_GUA = X1_585_new_GUA[['GUA_San_Diego']]
X1_predict_585_GUA = X1_585_new_GUA.drop(columns='GUA_San_Diego')

In [ ]:
print(X1_predict_119_GUA.isnull().sum())
print(X1_predict_119_GUA.isin([np.nan, np.inf, -np.inf]).sum())


# Training on TRAC: GUA_San_Diego

In [ ]:
# # Training on TRAC: Crossing 1: GUA_San_Diego

# _SSP119_avg_temp
model1_ssp119_GUA = sm.OLS(Y1_119_GUA, X1_predict_119_GUA).fit()
# _SSP245_avg_temp
model2_ssp245_GUA = sm.OLS(Y1_245_GUA, X1_predict_245_GUA).fit()
# _SSP370_avg_temp
model3_ssp370_GUA = sm.OLS(Y1_370_GUA, X1_predict_370_GUA).fit()
# _SSP585_avg_temp
model4_ssp585_GUA = sm.OLS(Y1_585_GUA, X1_predict_585_GUA).fit()

# Saving Model Summaries

models_and_data = [
    (model1_ssp119_GUA, "Y1_119_GUA", "X1_predict_119_GUA"),
    (model2_ssp245_GUA, "Y1_245_GUA", "X1_predict_245_GUA"),
    (model3_ssp370_GUA, "Y1_370_GUA", "X1_predict_370_GUA"),
    (model4_ssp585_GUA, "Y1_585_GUA", "X1_predict_585_GUA")
]

directory_path = "All_outputs/Guatemala/"

# Iterate over models and save their summaries
for model, Y_var, X_var in models_and_data:
    fitted_model = sm.OLS(eval(Y_var), eval(X_var)).fit()
    model_name = [name for name, obj in globals().items() if obj is model][0]
    model_name = f"Guatemala_San_Diego_{model_name}"
    file_path = f"{directory_path}{model_name}_summary.txt"
    # Save the model summary as a text file
    with open(file_path, "w") as file:
        file.write(str(fitted_model.summary()))

In [ ]:
# TRAC prediction 2023 to 2100: _SSP119_avg_temp
predicted_future_imigrants_ssp119_GUA = model1_ssp119_GUA.predict(GUA_new_features_2023_2100_ssp119)
# TRAC prediction 2023 to 2100: _SSP245_avg_temp
predicted_future_imigrants_ssp245_GUA = model2_ssp245_GUA.predict(GUA_new_features_2023_2100_ssp245)
# TRAC prediction 2023 to 2100: _SSP370_avg_temp
predicted_future_imigrants_ssp370_GUA = model3_ssp370_GUA.predict(GUA_new_features_2023_2100_ssp370)
# TRAC prediction 2023 to 2100: _SSP585_avg_temp
predicted_future_imigrants_ssp585_GUA = model4_ssp585_GUA.predict(GUA_new_features_2023_2100_ssp585)

In [ ]:
# Printing Dataframe for SSP119
GUA_new_features_2023_2100_ssp119['GUA_San_Diego'] = predicted_future_imigrants_ssp119_GUA
# Printing Dataframe for SSP245
GUA_new_features_2023_2100_ssp245['GUA_San_Diego'] = predicted_future_imigrants_ssp245_GUA
# Printing Dataframe for SSP370
GUA_new_features_2023_2100_ssp370['GUA_San_Diego'] = predicted_future_imigrants_ssp370_GUA
# Printing Dataframe for SSP585
GUA_new_features_2023_2100_ssp585['GUA_San_Diego'] = predicted_future_imigrants_ssp585_GUA

In [ ]:
##################################################################
# Removing duplicated data per year: _SSP119_avg_temp
years = GUA_new_features_2023_2100_ssp119['Year'].unique()
final_predictions_2023_2100_ssp119_GUA = pd.DataFrame()  
for year in years:
    first_row = GUA_new_features_2023_2100_ssp119[GUA_new_features_2023_2100_ssp119['Year'] == year].iloc[0]  
    final_predictions_2023_2100_ssp119_GUA = pd.concat([final_predictions_2023_2100_ssp119_GUA, pd.DataFrame([first_row])])
final_predictions_2023_2100_ssp119_GUA.reset_index(drop=True, inplace=True)
final_predictions_2023_2100_ssp119_GUA['Year'] = final_predictions_2023_2100_ssp119_GUA['Year'].astype(int)
##################################################################
# Removing duplicated data per year: _SSP245_avg_temp
years = GUA_new_features_2023_2100_ssp245['Year'].unique()
final_predictions_2023_2100_ssp245_GUA = pd.DataFrame()  
for year in years:
    first_row = GUA_new_features_2023_2100_ssp245[GUA_new_features_2023_2100_ssp245['Year'] == year].iloc[0]  
    final_predictions_2023_2100_ssp245_GUA = pd.concat([final_predictions_2023_2100_ssp245_GUA, pd.DataFrame([first_row])])
final_predictions_2023_2100_ssp245_GUA.reset_index(drop=True, inplace=True)
final_predictions_2023_2100_ssp245_GUA['Year'] = final_predictions_2023_2100_ssp245_GUA['Year'].astype(int)
##################################################################
# Removing duplicated data per year: _SSP370_avg_temp
years = GUA_new_features_2023_2100_ssp370['Year'].unique()
final_predictions_2023_2100_ssp370_GUA = pd.DataFrame()  
for year in years:
    first_row = GUA_new_features_2023_2100_ssp370[GUA_new_features_2023_2100_ssp370['Year'] == year].iloc[0]  
    final_predictions_2023_2100_ssp370_GUA = pd.concat([final_predictions_2023_2100_ssp370_GUA, pd.DataFrame([first_row])])
final_predictions_2023_2100_ssp370_GUA.reset_index(drop=True, inplace=True)
final_predictions_2023_2100_ssp370_GUA['Year'] = final_predictions_2023_2100_ssp370_GUA['Year'].astype(int)
##################################################################
# Removing duplicated data per year: _SSP585_avg_temp
years = GUA_new_features_2023_2100_ssp585['Year'].unique()
final_predictions_2023_2100_ssp585_GUA = pd.DataFrame()  
for year in years:
    first_row = GUA_new_features_2023_2100_ssp585[GUA_new_features_2023_2100_ssp585['Year'] == year].iloc[0]  
    final_predictions_2023_2100_ssp585_GUA = pd.concat([final_predictions_2023_2100_ssp585_GUA, pd.DataFrame([first_row])])
final_predictions_2023_2100_ssp585_GUA.reset_index(drop=True, inplace=True)
final_predictions_2023_2100_ssp585_GUA['Year'] = final_predictions_2023_2100_ssp585_GUA['Year'].astype(int)


# Plotting TRAC Predictions

<!-- final_predictions_2023_2100_ssp119
final_predictions_2023_2100_ssp245
final_predictions_2023_2100_ssp370
final_predictions_2023_2100_ssp585 -->

In [ ]:
# Setting the index of all DataFrames to 'Year'
final_predictions_2023_2100_ssp119_GUA.set_index('Year', inplace=True)
final_predictions_2023_2100_ssp245_GUA.set_index('Year', inplace=True)
final_predictions_2023_2100_ssp370_GUA.set_index('Year', inplace=True)
final_predictions_2023_2100_ssp585_GUA.set_index('Year', inplace=True)
###########
# Plotting
plt.figure(figsize=(10, 5))  # You can adjust the figure size as needed
for ssp, label in zip([final_predictions_2023_2100_ssp119_GUA, final_predictions_2023_2100_ssp245_GUA, final_predictions_2023_2100_ssp370_GUA, final_predictions_2023_2100_ssp585_GUA], ['SSP119', 'SSP245', 'SSP370', 'SSP585']):
    plt.plot(ssp.index, ssp['GUA_San_Diego'], marker='o', linestyle='-', label=label)
###########
plt.title('San Diego Crossing: Proyection of Guatemalan Undocumented Immigrants from 2023 to 2100')
plt.xlabel('Year')
plt.ylabel('GUA_San_Diego')
plt.grid(True)
plt.legend()

plt.savefig("All_outputs/Guatemala/San_Diego_GUA_All_Projections.png")

plt.show()

In [ ]:
# Function to extract data for each scenario
def extract_data(final_predictions, scenario):
    return pd.DataFrame({
        'Scenario': [scenario] * len(final_predictions),
        'Year': final_predictions.index,
        'Number_of_People': final_predictions['GUA_San_Diego']
    })
# Extract data for each scenario
data_ssp119_GUA = extract_data(final_predictions_2023_2100_ssp119_GUA, 'SSP119')
data_ssp245_GUA = extract_data(final_predictions_2023_2100_ssp245_GUA, 'SSP245')
data_ssp370_GUA = extract_data(final_predictions_2023_2100_ssp370_GUA, 'SSP370')
data_ssp585_GUA = extract_data(final_predictions_2023_2100_ssp585_GUA, 'SSP585')
# Concatenating all scenarios into a single DataFrame
output_df_GUA = pd.concat([data_ssp119_GUA, data_ssp245_GUA, data_ssp370_GUA, data_ssp585_GUA], ignore_index=True)
# Pivot the DataFrame
output_df_pivoted_GUA = output_df_GUA.pivot(index='Scenario', columns='Year', values='Number_of_People')
# Fill any missing values with 0
output_df_pivoted_GUA.fillna(0, inplace=True)

# Reformatting dataframe
output_df_pivoted_GUA.insert(0, 'Crossing', 'San_Diego_Crossing')
output_df_pivoted_GUA.insert(1, 'Country', 'GUA')

# Saving dataframe

output_df_pivoted_GUA.to_csv("All_outputs/Guatemala/San_Diego_GUA_4SSP.csv", index=True)

# Printing
output_df_pivoted_GUA

# Final Plots

In [ ]:
# Plotting results

##################################################################
# Using ggplot
plt.style.use('ggplot')
##################################################################
# Color palette
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
##################################################################
# Plot size
plt.figure(figsize=(15, 5))  # You can adjust the figure size as needed
##################################################################
# Convert index values to strings
output_df_pivoted_GUA.index = output_df_pivoted_GUA.index.astype(str)
##################################################################
# Selecting every 5th year to display
years_to_display = output_df_pivoted_GUA.columns[2:][::5]  # Excluding first two columns
##################################################################
# Plotting data for each scenario
for scenario, color in zip(output_df_pivoted_GUA.index, colors):
    plt.plot(output_df_pivoted_GUA.columns[2:].astype(int), output_df_pivoted_GUA.loc[scenario][2:], marker='o', linestyle='-', alpha=0.5, label=scenario, color=color)
    for year in years_to_display:
        plt.annotate(f'{int(output_df_pivoted_GUA.loc[scenario, year])}', (int(year), output_df_pivoted_GUA.loc[scenario, year]), textcoords="offset points", xytext=(0,10), ha='center')
plt.title('San Diego Crossing: Undocumented Guatemalan Immigrants')
plt.suptitle('Projection from 2023 to 2100: SSP Scenarios')
plt.xlabel('Year')
plt.ylabel('People (units)')
##################################################################
# Linear Regression Equation
X = output_df_pivoted_GUA.columns[2:].astype(int).values.reshape(-1, 1)
for scenario, color in zip(output_df_pivoted_GUA.index, colors):
    y = output_df_pivoted_GUA.loc[scenario][2:].values.reshape(-1, 1)
    model = LinearRegression().fit(X, y)
    m = model.coef_[0][0]
    b = model.intercept_[0]
    plt.plot(output_df_pivoted_GUA.columns[2:].astype(int), m * output_df_pivoted_GUA.columns[2:].astype(int) + b, label=f'Linear Regression ({scenario}): y = {m:.2f}x + {b:.2f}', color=color)
plt.grid(True)
plt.legend()
plt.tight_layout()
##################################################################

# Saving graph
plt.savefig("All_outputs/Guatemala/San_Diego_GUA_4SSP_Single.png")

plt.show()